# Alert Agent

- Has a tool to send alerts (just prints in notebook)
- Decides whether to alert based on weather data

In [ ]:
from protolink.agents import Agent
from protolink.models import Message, Task
from protolink.transport import HTTPAgentTransport

URL = "http://localhost:8020"

# This could be an env variable
REGISTRY_URL = "http://localhost:9000"


# -------------------------
# Transport
# -------------------------
transport = HTTPAgentTransport(url=URL)


# -------------------------
# Task Handler
# -------------------------
class AlertAgent(Agent):
    # Override handle_task
    async def handle_task(self, task: Task):
        data = task.payload
        if data["temperature"] > 25:
            await self.call_tool("alert_tool", message=f"Hot weather in {data['city']}! {data['temperature']}°C")
        return task

    async def send_task_to(self, agent_url: str, task: Task) -> Task:
        """Send a task to another agent.

        Args:
            agent_url: URL of the target agent
            task: Task to send

        Returns:
            Task with response from target agent

        Raises:
            RuntimeError: If no transport is configured
        """
        if not self._client:
            raise RuntimeError("No transport client configured. Call set_transport() first.")
        return await self._client.send_task(agent_url, task)


# Define Agent Card using the a dict (Agent will cast it to AgentCard)
card = {"url": URL, "name": "AlertAgent", "description": "Sends alerts based on data"}

agent = AlertAgent(card=card, transport=transport, registry=REGISTRY_URL)

# -------------------------
# Tools
# -------------------------


# Add Native tool using the decorator
@agent.tool(name="alert_tool", description="Send an alert")
async def send_alert(message: str):
    print(f"ALERT: {message}")
    return {"status": "sent", "message": message}


await agent.start()

In [ ]:
task = Task.create(Message.user("Whats the temp in Geneva"))
res = await agent.send_task_to("http://localhost:8010", task)
print(res.to_dict())